In [4]:
import pandas as pd
train_file = 'blp25_hatespeech_subtask_1B_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1B_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1B_test.tsv'


In [5]:
#tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# Load train/val/test DataFrames
train_df = pd.read_csv(train_file, sep="\t")
dev_df = pd.read_csv(validation_file , sep="\t")
test_df = pd.read_csv(test_file, sep="\t")





In [14]:
l2id = {'None': 0, 'Society': 1, 'Organization': 2, 'Community': 3, 'Individual': 4}
id2l = {v: k for k, v in l2id.items()}


def clean_label(x):
    # handle missing or NaN → "None"
    if pd.isna(x) or x == 'None':
        return 'None'
    # already list-like e.g. ['Abusive']
    if isinstance(x, list):
        return x[0] if len(x) > 0 else 'None'
    # string cases like "[]" or "[Abusive]" or "[Political Hate]"
    x = x.strip("[]").strip()
    if x == "":
        return 'None'
    return x


def process_df(df):
    # Ensure labels are proper lists
    df["label"] = df["label"].apply(clean_label)
    df["label"] = df["label"].fillna("None")
    # Now create binary label
    df["toxic"] = df["label"].apply(lambda x: "None" if x == "None" else "Toxic")
    df["label_id"] = df["label"].map(l2id)

    

  

    return df

train_df = process_df(train_df)
dev_df  = process_df(dev_df)

train_df

,id,text,label,toxic,label_id
0,147963,ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...,None,None,0
1,214275,ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...,None,None,0
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Individual,Toxic,4
3,821985,চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...,None,None,0
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Individual,Toxic,4
...,...,...,...,...,...
35517,790325,তইওয়ানের এত ক্ষমতা হয়নি যে এক টুকরো জায়গা নষ্ট...,None,None,0
35518,328377,চুরের ঘরের চুর হালা,None,None,0
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,Individual,Toxic,4
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,Organization,Toxic,2


In [15]:

# Display the result
print(train_df[['text', 'toxic']].head())

                                                text  toxic
0  ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...   None
1  ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...   None
2          অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে  Toxic
3  চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...   None
4  এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...  Toxic


In [30]:
df_train = train_df

df_val = dev_df

df_val

,id,text,label,toxic,label_id
0,166449,ইন্ডিয়া কি মাছ ধরা বন্ধ রাখছেএক নদীতে দুইনীতি ...,Society,Toxic,1
1,267692,লক্ষ টাকা ঘুষ দিয়ে অযোগ্য আর দায়িত্বহীন মানস...,Organization,Toxic,2
2,184031,ওহা ভবনের দালাল,None,None,0
3,939131,আর কতো শিখবে আমার সোনার ছেলেরা এগুলো কে টাকা দ...,Individual,Toxic,4
4,210284,কি সাংঘাতিক ভাই রে তুই,Individual,Toxic,4
...,...,...,...,...,...
2507,350971,পুরোনো ইতিহাস তুলে ধরার জন্য সময় সংবাদ কে ধন্...,None,None,0
2508,539053,এই জন্যই আমাদের মেয়েরা কোরিয়া চলে যেতে চায়,None,None,0
2509,200314,এই শালা ইবলিশ এর বস এবলিশ এদের দেখে ভয় পায়,Individual,Toxic,4
2510,788171,আমি কিনে ফেলছি আই ফোন ১৪,None,None,0


In [31]:
# import pandas as pd

# # existing train and dev
# df_train = train_df.copy()
# df_val   = dev_df.copy()

# # split dev_df in half
# half = len(df_val) // 2
# dev_train = df_val.iloc[:half].reset_index(drop=True)
# dev_val   = df_val.iloc[half:].reset_index(drop=True)

# # add first half of dev to training set
# df_train = pd.concat([df_train, dev_train], ignore_index=True)

# # final validation set is the second half
# df_val = dev_val

# print("Train size:", len(df_train))
# print("Validation size:", len(df_val))



import pandas as pd

# existing train and dev
df_train = train_df.copy()
df_val   = dev_df.copy()

# --- 75/25 split of dev_df ---
n   = len(df_val)
cut = int(0.5 * n)  # floor to an integer

# (optional) shuffle to avoid order bias:
# df_val = df_val.sample(frac=1, random_state=42).reset_index(drop=True)

dev_train = df_val.iloc[:cut].reset_index(drop=True)  # 75%
dev_val   = df_val.iloc[cut:].reset_index(drop=True)  # 25%

# add 75% of dev to training set
df_train = pd.concat([df_train, dev_train], ignore_index=True)

# final validation set is the remaining 25%
df_val = dev_val

print("Train size:", len(df_train))
print("Validation size:", len(df_val))


Train size: 36778
Validation size: 1256


In [32]:
df_train

,id,text,label,toxic,label_id
0,147963,ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...,None,None,0
1,214275,ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...,None,None,0
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Individual,Toxic,4
3,821985,চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...,None,None,0
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Individual,Toxic,4
...,...,...,...,...,...
36773,25078,জার্মান কুত্তা গুলো ভয় পাইছে দৌড়ঁ শুরু,Community,Toxic,3
36774,540478,শালার বাংগালি তোদের পুটানি বেশীপকিন্নি কোথাকার,Society,Toxic,1
36775,601373,মিথ্যাবাীদ খুনি হাসিনা,Individual,Toxic,4
36776,295630,সময় টিভি যে একটা দালাল হাসিনার পাছাটা কুত্তা ...,Organization,Toxic,2


In [18]:
toxic_df = df_train
target_list = sorted(toxic_df['toxic'].unique().tolist()) # Sort for consistent column order
print(f"Target Categories: {target_list}")

Target Categories: ['None', 'Toxic']


In [19]:
df_train = pd.get_dummies(df_train, columns=['toxic'], prefix='', prefix_sep='')[['text'] + target_list]

df_val = pd.get_dummies(df_val, columns=['toxic'], prefix='', prefix_sep='')[['text'] + target_list]

df_val

,text,None,Toxic
0,মিথ্যা নাটক করে লাভ নেই সাজানো নাটক শুরু হয়ে গেছে,False,True
1,ঘন্টা বাজাতে দফতরের কাজ টা কি আপনি নেবেন নাকি,True,False
2,ইতিহাসের দীর্ঘতম বিসিএস শেষে মেধাবীদের সাথে এভ...,False,True
3,সময় টিভি আসলে কি চায় ভারত সিদ্ধান্ত মেনে নিলে ...,False,True
4,আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি,False,True
...,...,...,...
1251,পুরোনো ইতিহাস তুলে ধরার জন্য সময় সংবাদ কে ধন্...,True,False
1252,এই জন্যই আমাদের মেয়েরা কোরিয়া চলে যেতে চায়,True,False
1253,এই শালা ইবলিশ এর বস এবলিশ এদের দেখে ভয় পায়,False,True
1254,আমি কিনে ফেলছি আই ফোন ১৪,True,False


In [20]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoConfig, AutoModel
from collections import defaultdict
from tqdm import tqdm as tq
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoConfig, AutoModel
from torch.optim import AdamW  # <-- use this instead


MODEL_NAME ="csebuetnlp/banglabert"
# Use a pipeline as a high-level helper
#MODEL_NAME = "google-bert/bert-base-multilingual-cased"

#MODEL_NAME="FacebookAI/xlm-roberta-large"

# You can keep these as they are or tune them
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
EPOCHS = 7
LEARNING_RATE = 2e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from collections import defaultdict
from tqdm import tqdm as tq
from sklearn.model_selection import train_test_split

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        # Corrected column name from 'title' or 'Text' to 'text'
        self.texts = list(df['text']) 
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'text': text
        }

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = CustomDataset(df_train, tokenizer, MAX_LEN, target_list)
val_dataset = CustomDataset(df_val, tokenizer, MAX_LEN, target_list)

train_data_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)


In [28]:
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm import tqdm as tq

# Your existing configuration
MODEL_NAME = "csebuetnlp/banglabert"
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
EPOCHS = 7
LEARNING_RATE = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TACT specific hyperparameters
EPSILON = 0.01  # Perturbation factor
LAMBDA = 0.3    # Balance parameter between BCE and InfoNCE
TEMPERATURE = 0.07  # Temperature for InfoNCE loss

class TACTBanglaBERT(nn.Module):
    def __init__(self, model_name, num_classes, hidden_dim=768):
        super(TACTBanglaBERT, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        
        # Multi-label classification head
        self.classifier = nn.Linear(hidden_dim, num_classes)
        
        # Non-linear projection layer for contrastive learning
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 128)  # Final projection dimension
        )
        
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        
    def forward(self, input_ids, attention_mask, token_type_ids=None, targets=None, apply_tact=True):
        # Get BERT embeddings
        outputs = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True
        )
        
        # Use [CLS] token representation from last hidden state
        # BanglaBERT doesn't have pooler_output, so we use the [CLS] token (first token)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state[:, 0, :]  # [CLS] token is at index 0
        
        if apply_tact and targets is not None and self.training:
            return self.forward_with_tact(pooled_output, targets)
        else:
            # Standard forward pass
            pooled_output = self.dropout(pooled_output)
            logits = self.classifier(pooled_output)
            
            loss = None
            if targets is not None:
                # Multi-label binary cross entropy
                loss = F.binary_cross_entropy_with_logits(logits, targets)
                
            return {
                "loss": loss, 
                "logits": logits, 
                "hidden_states": pooled_output
            }
    
    def forward_with_tact(self, original_repr, targets):
        """
        Implements TACT training with token-level adversarial perturbations
        """
        # Enable gradients for token representations
        original_repr = original_repr.requires_grad_(True)
        
        # Initial forward pass to compute gradients
        pooled_output = self.dropout(original_repr)
        logits = self.classifier(pooled_output)
        
        # Compute binary cross-entropy loss
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets)
        
        # Compute gradients w.r.t. token representations
        grad_outputs = torch.ones_like(bce_loss)
        grad = torch.autograd.grad(
            outputs=bce_loss,
            inputs=original_repr,
            grad_outputs=grad_outputs,
            create_graph=True,
            retain_graph=True,
            only_inputs=True
        )[0]
        
        # Calculate adversarial perturbation (Equation 5 from TACT paper)
        grad_norm = torch.norm(grad, dim=-1, keepdim=True)
        grad_norm = torch.clamp(grad_norm, min=1e-8)  # Avoid division by zero
        perturbation = -EPSILON * (grad / grad_norm)
        
        # Generate perturbed representation (Equation 6)
        perturbed_repr = original_repr + perturbation
        
        # Forward pass with original representation
        pooled_original = self.dropout(original_repr)
        logits_original = self.classifier(pooled_original)
        proj_original = self.projection(pooled_original)
        
        # Forward pass with perturbed representation  
        pooled_perturbed = self.dropout(perturbed_repr)
        logits_perturbed = self.classifier(pooled_perturbed)
        proj_perturbed = self.projection(pooled_perturbed)
        
        # Compute losses
        bce_loss_original = F.binary_cross_entropy_with_logits(logits_original, targets)
        bce_loss_perturbed = F.binary_cross_entropy_with_logits(logits_perturbed, targets)
        
        # InfoNCE Loss for contrastive learning
        infonce_loss = self.compute_infonce_loss(proj_original, proj_perturbed)
        
        # Combined loss (Equation 7 from TACT paper)
        total_loss = (1 - LAMBDA) / 2 * (bce_loss_original + bce_loss_perturbed) + LAMBDA * infonce_loss
        
        return {
            "loss": total_loss,
            "logits": logits_original,
            "hidden_states": pooled_original,
            "bce_loss_original": bce_loss_original,
            "bce_loss_perturbed": bce_loss_perturbed,
            "infonce_loss": infonce_loss
        }
    
    def compute_infonce_loss(self, z1, z2):
        """
        Compute InfoNCE loss for contrastive learning
        """
        batch_size = z1.size(0)
        
        # Normalize embeddings
        z1_norm = F.normalize(z1, dim=-1)
        z2_norm = F.normalize(z2, dim=-1)
        
        # Concatenate representations
        representations = torch.cat([z1_norm, z2_norm], dim=0)
        
        # Compute similarity matrix
        similarity_matrix = torch.matmul(representations, representations.T) / TEMPERATURE
        
        # Create positive pair labels
        labels = torch.cat([torch.arange(batch_size, 2*batch_size), 
                           torch.arange(0, batch_size)]).to(device)
        
        # Mask diagonal (self-similarity)
        mask = torch.eye(2*batch_size, dtype=torch.bool, device=device)
        similarity_matrix.masked_fill_(mask, -float('inf'))
        
        # InfoNCE loss
        loss = F.cross_entropy(similarity_matrix, labels)
        
        return loss

class TACTTrainer:
    def __init__(self, model, train_loader, val_loader, optimizer, scheduler=None):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_accuracy = 0.0
        
    def train_epoch(self):
        self.model.train()
        
        total_loss = 0
        total_bce_original = 0
        total_bce_perturbed = 0
        total_infonce = 0
        num_batches = 0
        
        progress_bar = tq(self.train_loader, desc="Training")
        
        for batch in progress_bar:
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            targets = batch['targets'].to(device)
            
            # Zero gradients
            self.optimizer.zero_grad()
            
            # Forward pass with TACT
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                targets=targets,
                apply_tact=True
            )
            
            loss = outputs['loss']
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.optimizer.step()
            if self.scheduler:
                self.scheduler.step()
            
            # Update metrics
            total_loss += loss.item()
            if 'bce_loss_original' in outputs:
                total_bce_original += outputs['bce_loss_original'].item()
            if 'bce_loss_perturbed' in outputs:
                total_bce_perturbed += outputs['bce_loss_perturbed'].item()
            if 'infonce_loss' in outputs:
                total_infonce += outputs['infonce_loss'].item()
            
            num_batches += 1
            
            # Update progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'LR': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })
        
        # Calculate averages
        metrics = {
            'avg_loss': total_loss / num_batches,
            'avg_bce_original': total_bce_original / num_batches,
            'avg_bce_perturbed': total_bce_perturbed / num_batches,
            'avg_infonce': total_infonce / num_batches
        }
        
        return metrics
    
    def evaluate(self):
        self.model.eval()
        
        total_loss = 0
        all_predictions = []
        all_targets = []
        num_batches = 0
        
        progress_bar = tq(self.val_loader, desc="Validation")
        
        with torch.no_grad():
            for batch in progress_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                targets = batch['targets'].to(device)
                
                # Forward pass without TACT
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    targets=targets,
                    apply_tact=False
                )
                
                loss = outputs['loss']
                total_loss += loss.item()
                num_batches += 1
                
                # Get predictions
                logits = outputs['logits']
                predictions = torch.sigmoid(logits)
                predicted_labels = (predictions > 0.5).float()
                
                all_predictions.extend(predicted_labels.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                
                progress_bar.set_postfix({'Val Loss': f'{loss.item():.4f}'})
        
        # Calculate metrics
        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)
        
        # Exact match accuracy (all labels must match)
        exact_match_accuracy = accuracy_score(all_targets, all_predictions)
        
        # Per-label accuracy
        label_accuracies = []
        for i in range(all_targets.shape[1]):
            label_acc = accuracy_score(all_targets[:, i], all_predictions[:, i])
            label_accuracies.append(label_acc)
        
        avg_loss = total_loss / num_batches
        
        return avg_loss, exact_match_accuracy, label_accuracies

def train_tact_model(train_data_loader, val_data_loader, target_list, num_epochs=EPOCHS):
    """
    Train TACT model using your existing data loaders
    """
    print(f"Training TACT model with {len(target_list)} labels: {target_list}")
    print(f"Device: {device}")
    
    # Initialize model
    model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
    
    # Print model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    # Optimizer with weight decay
    optimizer = AdamW(
        model.parameters(), 
        lr=LEARNING_RATE, 
        weight_decay=0.01,
        eps=1e-8
    )
    
    # Learning rate scheduler
    total_steps = len(train_data_loader) * num_epochs
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, 
        start_factor=1.0, 
        end_factor=0.1, 
        total_iters=total_steps
    )
    
    # Initialize trainer
    trainer = TACTTrainer(model, train_data_loader, val_data_loader, optimizer, scheduler)
    
    print(f"\nStarting training for {num_epochs} epochs...")
    print("=" * 80)
    
    # Training loop
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 60)
        
        # Train
        train_metrics = trainer.train_epoch()
        print(f"Training - Loss: {train_metrics['avg_loss']:.4f}, "
              f"BCE Orig: {train_metrics['avg_bce_original']:.4f}, "
              f"BCE Pert: {train_metrics['avg_bce_perturbed']:.4f}, "
              f"InfoNCE: {train_metrics['avg_infonce']:.4f}")
        
        # Evaluate
        val_loss, exact_accuracy, label_accuracies = trainer.evaluate()
        print(f"Validation - Loss: {val_loss:.4f}, Exact Match Accuracy: {exact_accuracy:.4f}")
        
        # Print per-label accuracies
        print("Per-label accuracies:")
        for i, (label_name, acc) in enumerate(zip(target_list, label_accuracies)):
            print(f"  {label_name}: {acc:.4f}")
        
        # Save best model
        if exact_accuracy > best_accuracy:
            best_accuracy = exact_accuracy
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_accuracy': best_accuracy,
                'target_list': target_list
            }, 'best_tact_bangla_model.pth')
            print(f"✓ New best model saved! Accuracy: {best_accuracy:.4f}")
    
    print(f"\nTraining completed! Best exact match accuracy: {best_accuracy:.4f}")
    return model, trainer

# Inference function for your trained model
def predict_with_tact(model, text, tokenizer, target_list, max_len=MAX_LEN, threshold=0.5):
    """
    Make predictions using the trained TACT model
    """
    model.eval()
    
    # Tokenize text
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Move to device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            apply_tact=False
        )
        
        # Get probabilities
        logits = outputs['logits']
        probabilities = torch.sigmoid(logits).cpu().numpy()[0]
        
        # Create result dictionary
        results = {}
        for i, (label, prob) in enumerate(zip(target_list, probabilities)):
            results[label] = {
                'probability': float(prob),
                'predicted': bool(prob > threshold)
            }
    
    return results

# Usage with your existing setup:
"""
# Assuming you already have:
# - df_train, df_val dataframes
# - target_list (e.g., ['Community', 'Individual', 'None', 'Organization', 'Society'])
# - train_data_loader, val_data_loader from your CustomDataset

# Train the TACT model
model, trainer = train_tact_model(train_data_loader, val_data_loader, target_list)

# Make predictions
sample_text = "আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি"
predictions = predict_with_tact(model, sample_text, tokenizer, target_list)
print(predictions)

# Load saved model for inference
checkpoint = torch.load('best_tact_bangla_model.pth')
model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
"""

# Usage with your existing setup:

# Assuming you already have:
# - df_train, df_val dataframes
# - target_list (e.g., ['Community', 'Individual', 'None', 'Organization', 'Society'])
# - train_data_loader, val_data_loader from your CustomDataset

# Train the TACT model
model, trainer = train_tact_model(train_data_loader, val_data_loader, target_list)

# Make predictions
# sample_text = "আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি"
# predictions = predict_with_tact(model, sample_text, tokenizer, target_list)
# print(predictions)

# # Load saved model for inference
# checkpoint = torch.load('best_tact_bangla_model.pth')
# model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
# model.load_state_dict(checkpoint['model_state_dict'])

Training TACT model with 2 labels: ['None', 'Toxic']
Device: cuda
Total parameters: 110,422,210
Trainable parameters: 110,422,210

Starting training for 7 epochs...

Epoch 1/7
------------------------------------------------------------


Training: 100%|██████████| 2299/2299 [08:51<00:00,  4.33it/s, Loss=0.2850, LR=1.74e-05]


Training - Loss: 0.3608, BCE Orig: 0.4818, BCE Pert: 0.4804, InfoNCE: 0.0802


Validation: 100%|██████████| 79/79 [00:06<00:00, 12.61it/s, Val Loss=0.4676]


Validation - Loss: 0.4584, Exact Match Accuracy: 0.7763
Per-label accuracies:
  None: 0.7818
  Toxic: 0.7866
✓ New best model saved! Accuracy: 0.7763

Epoch 2/7
------------------------------------------------------------


...

Validation - Loss: 1.0672, Exact Match Accuracy: 0.7739
Per-label accuracies:
  None: 0.7747
  Toxic: 0.7747

Epoch 7/7
------------------------------------------------------------


Training: 100%|██████████| 2299/2299 [09:06<00:00,  4.21it/s, Loss=0.0059, LR=2.00e-06]


Training - Loss: 0.0415, BCE Orig: 0.0565, BCE Pert: 0.0561, InfoNCE: 0.0068


Validation: 100%|██████████| 79/79 [00:06<00:00, 12.61it/s, Val Loss=1.6532]

Validation - Loss: 1.2157, Exact Match Accuracy: 0.7755
Per-label accuracies:
  None: 0.7755
  Toxic: 0.7771

Training completed! Best exact match accuracy: 0.7882


In [40]:
df_train = df_train[df_train['label']!='None']
df_train

,id,text,label,toxic,label_id
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Individual,Toxic,4
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Individual,Toxic,4
7,786609,ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...,Society,Toxic,1
11,124917,মুসলিম বাচ্চাগুলো বাচ্চা পেরে পেরে গোটা পৃথিবী...,Community,Toxic,3
15,432369,বাড়ি বাল ফালায়ছে,Individual,Toxic,4
...,...,...,...,...,...
36773,25078,জার্মান কুত্তা গুলো ভয় পাইছে দৌড়ঁ শুরু,Community,Toxic,3
36774,540478,শালার বাংগালি তোদের পুটানি বেশীপকিন্নি কোথাকার,Society,Toxic,1
36775,601373,মিথ্যাবাীদ খুনি হাসিনা,Individual,Toxic,4
36776,295630,সময় টিভি যে একটা দালাল হাসিনার পাছাটা কুত্তা ...,Organization,Toxic,2


In [37]:
target_list1  = target_list

In [41]:
toxic_df = df_train
target_list = sorted(toxic_df['label'].unique().tolist()) # Sort for consistent column order
print(f"Target Categories: {target_list}")

Target Categories: ['Community', 'Individual', 'Organization', 'Society']


In [42]:
df_train = pd.get_dummies(df_train, columns=['label'], prefix='', prefix_sep='')[['text'] + target_list]

df_val = pd.get_dummies(df_val, columns=['label'], prefix='', prefix_sep='')[['text'] + target_list]

df_val

,text,Community,Individual,Organization,Society
0,মিথ্যা নাটক করে লাভ নেই সাজানো নাটক শুরু হয়ে গেছে,False,True,False,False
1,ঘন্টা বাজাতে দফতরের কাজ টা কি আপনি নেবেন নাকি,False,False,False,False
2,ইতিহাসের দীর্ঘতম বিসিএস শেষে মেধাবীদের সাথে এভ...,False,False,True,False
3,সময় টিভি আসলে কি চায় ভারত সিদ্ধান্ত মেনে নিলে ...,False,False,True,False
4,আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি,False,False,True,False
...,...,...,...,...,...
1251,পুরোনো ইতিহাস তুলে ধরার জন্য সময় সংবাদ কে ধন্...,False,False,False,False
1252,এই জন্যই আমাদের মেয়েরা কোরিয়া চলে যেতে চায়,False,False,False,False
1253,এই শালা ইবলিশ এর বস এবলিশ এদের দেখে ভয় পায়,False,True,False,False
1254,আমি কিনে ফেলছি আই ফোন ১৪,False,False,False,False


In [43]:

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from collections import defaultdict
from tqdm import tqdm as tq
from sklearn.model_selection import train_test_split

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        # Corrected column name from 'title' or 'Text' to 'text'
        self.texts = list(df['text']) 
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index]),
            'text': text
        }

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = CustomDataset(df_train, tokenizer, MAX_LEN, target_list)
val_dataset = CustomDataset(df_val, tokenizer, MAX_LEN, target_list)

train_data_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)


In [46]:
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm import tqdm as tq

# Your existing configuration
MODEL_NAME = "csebuetnlp/banglabert"
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VAL_BATCH_SIZE = 16
EPOCHS = 7
LEARNING_RATE = 2e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TACT specific hyperparameters
EPSILON = 0.01  # Perturbation factor
LAMBDA = 0.3    # Balance parameter between BCE and InfoNCE
TEMPERATURE = 0.07  # Temperature for InfoNCE loss

class TACTBanglaBERT(nn.Module):
    def __init__(self, model_name, num_classes, hidden_dim=768):
        super(TACTBanglaBERT, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        
        # Multi-label classification head
        self.classifier = nn.Linear(hidden_dim, num_classes)
        
        # Non-linear projection layer for contrastive learning
        self.projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, 128)  # Final projection dimension
        )
        
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        
    def forward(self, input_ids, attention_mask, token_type_ids=None, targets=None, apply_tact=True):
        # Get BERT embeddings
        outputs = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_hidden_states=True
        )
        
        # Use [CLS] token representation from last hidden state
        # BanglaBERT doesn't have pooler_output, so we use the [CLS] token (first token)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = last_hidden_state[:, 0, :]  # [CLS] token is at index 0
        
        if apply_tact and targets is not None and self.training:
            return self.forward_with_tact(pooled_output, targets)
        else:
            # Standard forward pass
            pooled_output = self.dropout(pooled_output)
            logits = self.classifier(pooled_output)
            
            loss = None
            if targets is not None:
                # Multi-label binary cross entropy
                loss = F.binary_cross_entropy_with_logits(logits, targets)
                
            return {
                "loss": loss, 
                "logits": logits, 
                "hidden_states": pooled_output
            }
    
    def forward_with_tact(self, original_repr, targets):
        """
        Implements TACT training with token-level adversarial perturbations
        """
        # Enable gradients for token representations
        original_repr = original_repr.requires_grad_(True)
        
        # Initial forward pass to compute gradients
        pooled_output = self.dropout(original_repr)
        logits = self.classifier(pooled_output)
        
        # Compute binary cross-entropy loss
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets)
        
        # Compute gradients w.r.t. token representations
        grad_outputs = torch.ones_like(bce_loss)
        grad = torch.autograd.grad(
            outputs=bce_loss,
            inputs=original_repr,
            grad_outputs=grad_outputs,
            create_graph=True,
            retain_graph=True,
            only_inputs=True
        )[0]
        
        # Calculate adversarial perturbation (Equation 5 from TACT paper)
        grad_norm = torch.norm(grad, dim=-1, keepdim=True)
        grad_norm = torch.clamp(grad_norm, min=1e-8)  # Avoid division by zero
        perturbation = -EPSILON * (grad / grad_norm)
        
        # Generate perturbed representation (Equation 6)
        perturbed_repr = original_repr + perturbation
        
        # Forward pass with original representation
        pooled_original = self.dropout(original_repr)
        logits_original = self.classifier(pooled_original)
        proj_original = self.projection(pooled_original)
        
        # Forward pass with perturbed representation  
        pooled_perturbed = self.dropout(perturbed_repr)
        logits_perturbed = self.classifier(pooled_perturbed)
        proj_perturbed = self.projection(pooled_perturbed)
        
        # Compute losses
        bce_loss_original = F.binary_cross_entropy_with_logits(logits_original, targets)
        bce_loss_perturbed = F.binary_cross_entropy_with_logits(logits_perturbed, targets)
        
        # InfoNCE Loss for contrastive learning
        infonce_loss = self.compute_infonce_loss(proj_original, proj_perturbed)
        
        # Combined loss (Equation 7 from TACT paper)
        total_loss = (1 - LAMBDA) / 2 * (bce_loss_original + bce_loss_perturbed) + LAMBDA * infonce_loss
        
        return {
            "loss": total_loss,
            "logits": logits_original,
            "hidden_states": pooled_original,
            "bce_loss_original": bce_loss_original,
            "bce_loss_perturbed": bce_loss_perturbed,
            "infonce_loss": infonce_loss
        }
    
    def compute_infonce_loss(self, z1, z2):
        """
        Compute InfoNCE loss for contrastive learning
        """
        batch_size = z1.size(0)
        
        # Normalize embeddings
        z1_norm = F.normalize(z1, dim=-1)
        z2_norm = F.normalize(z2, dim=-1)
        
        # Concatenate representations
        representations = torch.cat([z1_norm, z2_norm], dim=0)
        
        # Compute similarity matrix
        similarity_matrix = torch.matmul(representations, representations.T) / TEMPERATURE
        
        # Create positive pair labels
        labels = torch.cat([torch.arange(batch_size, 2*batch_size), 
                           torch.arange(0, batch_size)]).to(device)
        
        # Mask diagonal (self-similarity)
        mask = torch.eye(2*batch_size, dtype=torch.bool, device=device)
        similarity_matrix.masked_fill_(mask, -float('inf'))
        
        # InfoNCE loss
        loss = F.cross_entropy(similarity_matrix, labels)
        
        return loss

class TACTTrainer:
    def __init__(self, model, train_loader, val_loader, optimizer, scheduler=None):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_accuracy = 0.0
        
    def train_epoch(self):
        self.model.train()
        
        total_loss = 0
        total_bce_original = 0
        total_bce_perturbed = 0
        total_infonce = 0
        num_batches = 0
        
        progress_bar = tq(self.train_loader, desc="Training")
        
        for batch in progress_bar:
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            targets = batch['targets'].to(device)
            
            # Zero gradients
            self.optimizer.zero_grad()
            
            # Forward pass with TACT
            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                targets=targets,
                apply_tact=True
            )
            
            loss = outputs['loss']
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.optimizer.step()
            if self.scheduler:
                self.scheduler.step()
            
            # Update metrics
            total_loss += loss.item()
            if 'bce_loss_original' in outputs:
                total_bce_original += outputs['bce_loss_original'].item()
            if 'bce_loss_perturbed' in outputs:
                total_bce_perturbed += outputs['bce_loss_perturbed'].item()
            if 'infonce_loss' in outputs:
                total_infonce += outputs['infonce_loss'].item()
            
            num_batches += 1
            
            # Update progress bar
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'LR': f'{self.optimizer.param_groups[0]["lr"]:.2e}'
            })
        
        # Calculate averages
        metrics = {
            'avg_loss': total_loss / num_batches,
            'avg_bce_original': total_bce_original / num_batches,
            'avg_bce_perturbed': total_bce_perturbed / num_batches,
            'avg_infonce': total_infonce / num_batches
        }
        
        return metrics
    
    def evaluate(self):
        self.model.eval()
        
        total_loss = 0
        all_predictions = []
        all_targets = []
        num_batches = 0
        
        progress_bar = tq(self.val_loader, desc="Validation")
        
        with torch.no_grad():
            for batch in progress_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                targets = batch['targets'].to(device)
                
                # Forward pass without TACT
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    targets=targets,
                    apply_tact=False
                )
                
                loss = outputs['loss']
                total_loss += loss.item()
                num_batches += 1
                
                # Get predictions
                logits = outputs['logits']
                predictions = torch.sigmoid(logits)
                predicted_labels = (predictions > 0.5).float()
                
                all_predictions.extend(predicted_labels.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                
                progress_bar.set_postfix({'Val Loss': f'{loss.item():.4f}'})
        
        # Calculate metrics
        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)
        
        # Exact match accuracy (all labels must match)
        exact_match_accuracy = accuracy_score(all_targets, all_predictions)
        
        # Per-label accuracy
        label_accuracies = []
        for i in range(all_targets.shape[1]):
            label_acc = accuracy_score(all_targets[:, i], all_predictions[:, i])
            label_accuracies.append(label_acc)
        
        avg_loss = total_loss / num_batches
        
        return avg_loss, exact_match_accuracy, label_accuracies

def train_tact_model(train_data_loader, val_data_loader, target_list, num_epochs=EPOCHS):
    """
    Train TACT model using your existing data loaders
    """
    print(f"Training TACT model with {len(target_list)} labels: {target_list}")
    print(f"Device: {device}")
    
    # Initialize model
    model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
    
    # Print model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    # Optimizer with weight decay
    optimizer = AdamW(
        model.parameters(), 
        lr=LEARNING_RATE, 
        weight_decay=0.01,
        eps=1e-8
    )
    
    # Learning rate scheduler
    total_steps = len(train_data_loader) * num_epochs
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, 
        start_factor=1.0, 
        end_factor=0.1, 
        total_iters=total_steps
    )
    
    # Initialize trainer
    trainer = TACTTrainer(model, train_data_loader, val_data_loader, optimizer, scheduler)
    
    print(f"\nStarting training for {num_epochs} epochs...")
    print("=" * 80)
    
    # Training loop
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 60)
        
        # Train
        train_metrics = trainer.train_epoch()
        print(f"Training - Loss: {train_metrics['avg_loss']:.4f}, "
              f"BCE Orig: {train_metrics['avg_bce_original']:.4f}, "
              f"BCE Pert: {train_metrics['avg_bce_perturbed']:.4f}, "
              f"InfoNCE: {train_metrics['avg_infonce']:.4f}")
        
        # Evaluate
        val_loss, exact_accuracy, label_accuracies = trainer.evaluate()
        print(f"Validation - Loss: {val_loss:.4f}, Exact Match Accuracy: {exact_accuracy:.4f}")
        
        # Print per-label accuracies
        print("Per-label accuracies:")
        for i, (label_name, acc) in enumerate(zip(target_list, label_accuracies)):
            print(f"  {label_name}: {acc:.4f}")
        
        # Save best model
        if exact_accuracy > best_accuracy:
            best_accuracy = exact_accuracy
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_accuracy': best_accuracy,
                'target_list': target_list
            }, 'best_tact_bangla_model2.pth')
            print(f"✓ New best model saved! Accuracy: {best_accuracy:.4f}")
    
    print(f"\nTraining completed! Best exact match accuracy: {best_accuracy:.4f}")
    return model, trainer

# Inference function for your trained model
def predict_with_tact(model, text, tokenizer, target_list, max_len=MAX_LEN, threshold=0.5):
    """
    Make predictions using the trained TACT model
    """
    model.eval()
    
    # Tokenize text
    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Move to device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            apply_tact=False
        )
        
        # Get probabilities
        logits = outputs['logits']
        probabilities = torch.sigmoid(logits).cpu().numpy()[0]
        
        # Create result dictionary
        results = {}
        for i, (label, prob) in enumerate(zip(target_list, probabilities)):
            results[label] = {
                'probability': float(prob),
                'predicted': bool(prob > threshold)
            }
    
    return results

# Usage with your existing setup:
"""
# Assuming you already have:
# - df_train, df_val dataframes
# - target_list (e.g., ['Community', 'Individual', 'None', 'Organization', 'Society'])
# - train_data_loader, val_data_loader from your CustomDataset

# Train the TACT model
model, trainer = train_tact_model(train_data_loader, val_data_loader, target_list)

# Make predictions
sample_text = "আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি"
predictions = predict_with_tact(model, sample_text, tokenizer, target_list)
print(predictions)

# Load saved model for inference
checkpoint = torch.load('best_tact_bangla_model.pth')
model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
"""

# Usage with your existing setup:

# Assuming you already have:
# - df_train, df_val dataframes
# - target_list (e.g., ['Community', 'Individual', 'None', 'Organization', 'Society'])
# - train_data_loader, val_data_loader from your CustomDataset

# Train the TACT model
model2, trainer2 = train_tact_model(train_data_loader, val_data_loader, target_list)

# Make predictions
# sample_text = "আমেরিকার হচ্ছে শয়তানের দাদর বাড়ি"
# predictions = predict_with_tact(model, sample_text, tokenizer, target_list)
# print(predictions)

# # Load saved model for inference
# checkpoint = torch.load('best_tact_bangla_model.pth')
# model = TACTBanglaBERT(MODEL_NAME, len(target_list)).to(device)
# model.load_state_dict(checkpoint['model_state_dict'])

Training TACT model with 4 labels: ['Community', 'Individual', 'Organization', 'Society']
Device: cuda
Total parameters: 110,423,748
Trainable parameters: 110,423,748

Starting training for 7 epochs...

Epoch 1/7
------------------------------------------------------------


Training: 100%|██████████| 928/928 [03:37<00:00,  4.26it/s, Loss=0.2249, LR=1.74e-05]


Training - Loss: 0.3588, BCE Orig: 0.4411, BCE Pert: 0.4412, InfoNCE: 0.1666


Validation: 100%|██████████| 79/79 [00:06<00:00, 11.68it/s, Val Loss=0.2532]


Validation - Loss: 0.3899, Exact Match Accuracy: 0.4164
Per-label accuracies:
  Community: 0.9005
  Individual: 0.6863
  Organization: 0.8846
  Society: 0.8774
✓ New best model saved! Accuracy: 0.4164

Epoch 2/7
------------------------------------------------------------


...

Validation - Loss: 0.6128, Exact Match Accuracy: 0.3018
Per-label accuracies:
  Community: 0.8328
  Individual: 0.6887
  Organization: 0.8161
  Society: 0.8503

Epoch 7/7
------------------------------------------------------------


Training: 100%|██████████| 928/928 [03:34<00:00,  4.33it/s, Loss=0.0846, LR=2.00e-06]


Training - Loss: 0.0746, BCE Orig: 0.0981, BCE Pert: 0.0978, InfoNCE: 0.0201


Validation: 100%|██████████| 79/79 [00:06<00:00, 12.62it/s, Val Loss=0.5873]

Validation - Loss: 0.6367, Exact Match Accuracy: 0.2922
Per-label accuracies:
  Community: 0.8384
  Individual: 0.7022
  Organization: 0.7930
  Society: 0.8392

Training completed! Best exact match accuracy: 0.4164


In [51]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer



# load checkpoint + target list
checkpoint1 = torch.load('best_tact_bangla_model.pth', map_location=device)
target_list1 = checkpoint1['target_list']
num_labels1 = len(target_list1)

In [52]:
num_labels1

2

In [53]:


# load checkpoint + target list
checkpoint2 = torch.load('best_tact_bangla_model2.pth', map_location=device)
target_list2 = checkpoint2['target_list']
num_labels2 = len(target_list2)

In [54]:
num_labels2

4

In [55]:
# inference dataset
class TestDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(
            str(self.texts[idx]),
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {k: v.squeeze(0) for k, v in enc.items()}

# dataloader
test_ds = TestDataset(test_df['text'].tolist(), tokenizer, MAX_LEN)
test_loader = DataLoader(test_ds, batch_size=VAL_BATCH_SIZE, shuffle=False)

In [49]:
# model1

NameError: name 'model1' is not defined

In [57]:

print("\n--- Evaluating Pipeline on Test Set ---")

model = TACTBanglaBERT(MODEL_NAME, num_labels1).to(device)
model.load_state_dict(checkpoint1['model_state_dict'])



model.eval()




--- Evaluating Pipeline on Test Set ---


TACTBanglaBERT(
  (bert): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [58]:
print("\n--- Evaluating Pipeline on Test Set ---")

model2 = TACTBanglaBERT(MODEL_NAME, num_labels2).to(device)
model2.load_state_dict(checkpoint2['model_state_dict'])



model2.eval()



--- Evaluating Pipeline on Test Set ---


TACTBanglaBERT(
  (bert): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=

In [ ]:
# from transformers import pipeline
# from sklearn.metrics import classification_report
# import numpy as np

# # --- 1. Load the fine-tuned models with truncation ---
# pipe_1 = pipeline(
#     "text-classification",
#     model="./final_model_task2_1",
#     tokenizer=tokenizer,
#     device=-1,   # force CPU
#     truncation=True,
#     max_length=512
# )

# pipe_2 = pipeline(
#     "text-classification",
#     model="./final_model_task2_2",
#     tokenizer=tokenizer,
#     device=-1,
#     truncation=True,
#     max_length=512,
#     return_all_scores=True
# )

# # --- 2. Run the pipeline on the test set ---
# predictions = []
# for text in test_df['text']:
#     # Model 1: Is it toxic?
#     result_1 = pipe_1(text, truncation=True, max_length=512)[0]
#     print(result_1)
#     is_toxic = 1 if result_1['label'] == 'LABEL_1' else 0
#     print(is_toxic)
    
#     pred_labels = [0] * pipe_2.model.config.num_labels  # default: all zero
    
#     if is_toxic:
#         # Model 2: Which toxic class? (highest score = predicted label)
#         result_2 = pipe_2(text, truncation=True, max_length=512)[0]
#         best_label = max(result_2, key=lambda x: x['score'])
#         pred_labels = [1 if r['label'] == best_label['label'] else 0 for r in result_2]
    
#     predictions.append(pred_labels)

# y_pred = np.array(predictions)
# print(y_pred.shape)


In [65]:
import torch
import numpy as np
import pandas as pd

def two_stage_classification(model1, model2, test_loader, device, 
                           model1_labels, model2_labels):
    """
    Simple two-stage classification:
    - Always run Model 1 (toxic vs none)
    - If Model 1 predicts 'toxic', run Model 2 (targeted group)
    - Return exact predictions from both models
    """
    
    model1.eval()
    model2.eval()
    
    all_results = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch.get('token_type_ids')
            
            if token_type_ids is None:
                token_type_ids = torch.zeros_like(input_ids, device=device)
            else:
                token_type_ids = token_type_ids.to(device)
            
            batch_size = input_ids.size(0)
            
            # Model 1 predictions
            out1 = model1(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                apply_tact=False
            )
            
            logits1 = out1['logits']
            probs1 = torch.sigmoid(logits1).cpu().numpy()
            
            # Get Model 1 predictions
            top1_idx_1 = probs1.argmax(axis=1)
            model1_pred_labels = [model1_labels[i] for i in top1_idx_1]
            model1_pred_probs = probs1.max(axis=1)
            
            # Process each sample in batch
            for i in range(batch_size):
                result = {
                    'model1_pred_label': model1_pred_labels[i],
                    'model1_pred_prob': model1_pred_probs[i]
                }
                
                # Add all Model 1 probabilities
                for j, label in enumerate(model1_labels):
                    result[f'model1_prob_{label}'] = probs1[i, j]
                
                # Run Model 2 if Model 1 predicts toxic
                if model1_pred_labels[i] != 'none':
                    # Single sample for Model 2
                    single_input_ids = input_ids[i:i+1]
                    single_attention_mask = attention_mask[i:i+1]
                    single_token_type_ids = token_type_ids[i:i+1]
                    
                    out2 = model2(
                        input_ids=single_input_ids,
                        attention_mask=single_attention_mask,
                        token_type_ids=single_token_type_ids,
                        apply_tact=False
                    )
                    
                    logits2 = out2['logits']
                    probs2 = torch.sigmoid(logits2).cpu().numpy()[0]
                    
                    # Get Model 2 predictions
                    top1_idx_2 = probs2.argmax()
                    model2_pred_label = model2_labels[top1_idx_2]
                    model2_pred_prob = probs2.max()
                    
                    result.update({
                        'model2_pred_label': model2_pred_label,
                        'model2_pred_prob': model2_pred_prob
                    })
                    
                    # Add all Model 2 probabilities
                    for j, label in enumerate(model2_labels):
                        result[f'model2_prob_{label}'] = probs2[j]
                        
                else:
                    # Model 1 predicted 'none', no Model 2 needed
                    result.update({
                        'model2_pred_label': None,
                        'model2_pred_prob': None
                    })
                    
                    # Set Model 2 probabilities to None
                    for label in model2_labels:
                        result[f'model2_prob_{label}'] = None
                
                all_results.append(result)
    
    return pd.DataFrame(all_results)

# Usage example
def run_classification():
    """
    Simple usage example
    """
    # Your models and data
    # model1 = your_toxic_vs_none_model
    # model2 = your_targeted_group_model  
    # test_loader = your_test_dataloader
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model1_labels = ['None', 'Toxic']
    model2_labels = ['Community', 'Individual', 'Organization', 'Society']
    
    # Run classification
    results_df = two_stage_classification(
        model1=model,
        model2=model2,
        test_loader=test_loader,
        device=device,
        model1_labels=model1_labels,
        model2_labels=model2_labels
    )
    
    # Save results
    results_df.to_csv('predictions.csv', index=False)
    
    print("Classification complete!")
    return results_df

if __name__ == "__main__":
    print("Simple Two-Stage Classification")
    print("Model 1: Toxic vs None")
    print("Model 2: Targeted Group (Community, Individual, None, Organization, Society)")
    results_df= run_classification()

Simple Two-Stage Classification
Model 1: Toxic vs None
Model 2: Targeted Group (Community, Individual, None, Organization, Society)
Classification complete!


In [66]:
results_df

,model1_pred_label,model1_pred_prob,model1_prob_None,model1_prob_Toxic,model2_pred_label,model2_pred_prob,model2_prob_Community,model2_prob_Individual,model2_prob_Organization,model2_prob_Society
0,Toxic,0.765944,0.223916,0.765944,Society,0.698820,0.212609,0.099830,0.162958,0.698820
1,None,0.869882,0.869882,0.152591,Individual,0.924347,0.041802,0.924347,0.036397,0.062195
2,Toxic,0.666546,0.326201,0.666546,Community,0.437921,0.437921,0.192385,0.191014,0.107692
3,Toxic,0.892045,0.098418,0.892045,Community,0.666176,0.666176,0.273572,0.064612,0.085645
4,Toxic,0.962215,0.041165,0.962215,Individual,0.872669,0.088213,0.872669,0.036446,0.098670
...,...,...,...,...,...,...,...,...,...,...
10195,Toxic,0.974814,0.021215,0.974814,Society,0.662861,0.197042,0.232197,0.065961,0.662861
10196,Toxic,0.906622,0.083939,0.906622,Individual,0.944971,0.042090,0.944971,0.033354,0.045933
10197,Toxic,0.709862,0.297250,0.709862,Individual,0.825428,0.093775,0.825428,0.053623,0.124087
10198,None,0.857183,0.857183,0.147416,Society,0.464831,0.197194,0.106125,0.191576,0.464831


In [59]:
# import numpy as np
# import pandas as pd


# #'Abusive', 'None', 'Political Hate', 'Profane', 'Religious Hate', 'Sexism'
# # #'Abusive', 'Political Hate', 'Profane', 'Religious Hate', 'Sexism']
# # 'Community', 'Individual', 'None', 'Organization', 'Society'
# # ['Community', 'Individual', 'None', 'Organization', 'Society']
# # Your mapping
# id2l = {
#     0: 'Community',
#     1: 'Individual',
#     2: 'None',
#     3: 'Organization',
#     4: 'Society'
   
# }

# # Example y_pred
# # y_pred = np.array([[0,0,0,0,0],[0,0,0,1,0],[1,0,0,0,0]])

# def decode_labels(row):
#     indices = np.where(row == 1)[0]
#     if len(indices) == 0:
#         return "None"
#     # If multiple labels, join them with comma
#     return ", ".join([id2l[i] for i in indices])

# # Convert predictions into a DataFrame column
# df = pd.DataFrame()
# df["Predicted_Label"] = [decode_labels(row) for row in y_pred]

# print(df.head())


In [69]:

df=pd.read_csv('predictions.csv')

df

,model1_pred_label,model1_pred_prob,model1_prob_None,model1_prob_Toxic,model2_pred_label,model2_pred_prob,model2_prob_Community,model2_prob_Individual,model2_prob_Organization,model2_prob_Society
0,Toxic,0.765945,0.223916,0.765945,Society,0.698820,0.212609,0.099830,0.162958,0.698820
1,NaN,0.869882,0.869882,0.152591,Individual,0.924347,0.041802,0.924347,0.036397,0.062195
2,Toxic,0.666546,0.326201,0.666546,Community,0.437921,0.437921,0.192385,0.191014,0.107692
3,Toxic,0.892045,0.098418,0.892045,Community,0.666176,0.666176,0.273572,0.064612,0.085645
4,Toxic,0.962215,0.041165,0.962215,Individual,0.872669,0.088213,0.872669,0.036446,0.098670
...,...,...,...,...,...,...,...,...,...,...
10195,Toxic,0.974814,0.021215,0.974814,Society,0.662861,0.197042,0.232197,0.065961,0.662861
10196,Toxic,0.906622,0.083939,0.906622,Individual,0.944971,0.042090,0.944971,0.033354,0.045933
10197,Toxic,0.709862,0.297250,0.709862,Individual,0.825428,0.093775,0.825428,0.053623,0.124087
10198,NaN,0.857183,0.857183,0.147416,Society,0.464831,0.197194,0.106125,0.191576,0.464831


In [70]:
m = df['model1_pred_label'].fillna('').str.casefold().eq('toxic')
df.loc[m, 'model1_pred_label'] = df.loc[m, 'model2_pred_label']


In [71]:
df

,model1_pred_label,model1_pred_prob,model1_prob_None,model1_prob_Toxic,model2_pred_label,model2_pred_prob,model2_prob_Community,model2_prob_Individual,model2_prob_Organization,model2_prob_Society
0,Society,0.765945,0.223916,0.765945,Society,0.698820,0.212609,0.099830,0.162958,0.698820
1,NaN,0.869882,0.869882,0.152591,Individual,0.924347,0.041802,0.924347,0.036397,0.062195
2,Community,0.666546,0.326201,0.666546,Community,0.437921,0.437921,0.192385,0.191014,0.107692
3,Community,0.892045,0.098418,0.892045,Community,0.666176,0.666176,0.273572,0.064612,0.085645
4,Individual,0.962215,0.041165,0.962215,Individual,0.872669,0.088213,0.872669,0.036446,0.098670
...,...,...,...,...,...,...,...,...,...,...
10195,Society,0.974814,0.021215,0.974814,Society,0.662861,0.197042,0.232197,0.065961,0.662861
10196,Individual,0.906622,0.083939,0.906622,Individual,0.944971,0.042090,0.944971,0.033354,0.045933
10197,Individual,0.709862,0.297250,0.709862,Individual,0.825428,0.093775,0.825428,0.053623,0.124087
10198,NaN,0.857183,0.857183,0.147416,Society,0.464831,0.197194,0.106125,0.191576,0.464831


In [72]:
test_df['label']=df['model1_pred_label']
test_df['model']='TACTBanglaBERT-hyerarchical'
test_df['label'].unique()

array(['Society', nan, 'Community', 'Individual', 'Organization'],
      dtype=object)

In [73]:
test_df = test_df.replace({np.nan: "None"}) 
test_df

,id,text,label,model
0,12764,ইজরায়েলের বিচার হওয়া উচিৎ,Society,TACTBanglaBERT-hyerarchical
1,202933,শামীম ওসামা বিন হাসিনা,None,TACTBanglaBERT-hyerarchical
2,165894,হেন কাপ পুলিশের মারে অন্যরা তাহলে পলিশের কি হব...,Community,TACTBanglaBERT-hyerarchical
3,124999,আল্লাহ্ এসব জানোয়ারদের শেষ করে দাও,Community,TACTBanglaBERT-hyerarchical
4,535301,ইহুদির বাচ্চা ইহুদী ই হবে,Individual,TACTBanglaBERT-hyerarchical
...,...,...,...,...
10195,908819,শালা জঙ্গি নিচু জাত বাংলাদেশের শালা নিজের দেশ ...,Society,TACTBanglaBERT-hyerarchical
10196,597085,এরে হেতি হাগল অই গেছে হেতিরে পাবনা নে,Individual,TACTBanglaBERT-hyerarchical
10197,901448,এখান থেকে প্রমান হলো বাংলার মীর জাফর চাইলেই তা...,Individual,TACTBanglaBERT-hyerarchical
10198,617821,শুনি যে এই দেশের সংবিধান গণতান্ত্রিক সংবিধান ক...,None,TACTBanglaBERT-hyerarchical


In [74]:
test_df = test_df[['id', 'label', 'model']]
test_df

,id,label,model
0,12764,Society,TACTBanglaBERT-hyerarchical
1,202933,None,TACTBanglaBERT-hyerarchical
2,165894,Community,TACTBanglaBERT-hyerarchical
3,124999,Community,TACTBanglaBERT-hyerarchical
4,535301,Individual,TACTBanglaBERT-hyerarchical
...,...,...,...
10195,908819,Society,TACTBanglaBERT-hyerarchical
10196,597085,Individual,TACTBanglaBERT-hyerarchical
10197,901448,Individual,TACTBanglaBERT-hyerarchical
10198,617821,None,TACTBanglaBERT-hyerarchical


In [75]:
test_df.to_csv("V7_subtask_2A.tsv", sep="\t", index=False)
print("Saved to final_ensemble.tsv")

Saved to final_ensemble.tsv
